In [224]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

In [187]:
jdata_sku_basic_info = pd.read_csv('jdata_sku_basic_info.csv')
jdata_user_action = pd.read_csv('jdata_user_action.csv')
jdata_user_basic_info = pd.read_csv('jdata_user_basic_info.csv')
jdata_user_comment_score = pd.read_csv('jdata_user_comment_score.csv')
jdata_user_order = pd.read_csv('jdata_user_order.csv')
user_purchase_time_gap = pd.read_csv('purchase_time_gap.csv')

In [190]:
jdata_user_basic_info.head()
#关联一下用户位置，用户全年下单次数最多区域定位用户常驻区域
ga =jdata_user_order.groupby(['user_id','o_area'])['sku_id'].count().to_frame()
gb = ga['sku_id'].groupby(level=0, group_keys=False)
usr_area_info =gb.nlargest(1).to_frame()

In [191]:
usr_area_info.reset_index(inplace=True)
usr_area_info.head()

,user_id,o_area,sku_id
0,1,14,6
1,2,27,1
2,3,26,18
3,4,8,1
4,5,8,1


In [192]:
jdata_user_info_all = pd.merge(jdata_user_basic_info,usr_area_info,on='user_id',how='left')

In [195]:
jdata_user_behavior = pd.merge(jdata_user_action,jdata_sku_basic_info,on='sku_id', how='left')
jdata_user_behavior.head()

,user_id,sku_id,a_date,a_num,a_type,price,cate,para_1,para_2,para_3
0,1,80036,2017-04-14,4,1,511.8,101,189.25,3,4
1,1,96959,2017-01-12,2,1,412.6,101,83.14,3,4
2,1,8017,2017-03-09,1,1,562.9,101,147.55,3,4
3,1,80036,2017-04-30,6,1,511.8,101,189.25,3,4
4,1,16607,2017-01-12,3,1,557.0,101,442.63,4,5


In [196]:
jdata_order_behavior = pd.merge(jdata_user_order,jdata_sku_basic_info,on='sku_id', how='left')
jdata_order_behavior.head()

,user_id,sku_id,o_id,o_date,o_area,o_sku_num,price,cate,para_1,para_2,para_3
0,1,80036,480066,2017-03-09,14,1,511.8,101,189.25,3,4
1,1,16607,106933,2017-01-12,14,1,557.0,101,442.63,4,5
2,1,80036,542604,2017-04-14,14,1,511.8,101,189.25,3,4
3,1,80036,196335,2017-04-30,14,1,511.8,101,189.25,3,4
4,1,80036,42502,2017-03-22,14,2,511.8,101,189.25,3,4


In [197]:
jdata_order_behavior['o_date'] = pd.to_datetime(jdata_order_behavior['o_date'])
jdata_user_behavior['a_date'] = pd.to_datetime(jdata_user_behavior['a_date'])
jdata_user_comment_score['comment_create_tm'] = pd.to_datetime(jdata_user_comment_score['comment_create_tm'])

jdata_order_behavior['o_month'] = jdata_order_behavior['o_date'].apply(lambda x: x.month)
jdata_user_behavior['a_month'] = jdata_user_behavior['a_date'].apply(lambda x: x.month)
jdata_user_comment_score['month'] = jdata_user_comment_score['comment_create_tm'].apply(lambda x: x.month)

In [199]:
#总订单数以及月平均订单数
data_whole_year = pd.DataFrame()
for month in [5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4]:
        mon_data = jdata_order_behavior[jdata_order_behavior['o_month'] == month].sort_values(by=['o_date', 'user_id']).drop_duplicates()
        data_whole_year = pd.concat([data_whole_year, mon_data])
##################### 生成用户年订单数 ####################
users_pd = pd.DataFrame()
g_users = data_whole_year.groupby(['user_id'])
all_users = g_users.groups.keys()
users_pd['user_id'] = all_users
users_pd['year_user_orders'] = g_users.agg(['count']).values[:,:1].flatten().tolist()

##################### 生成用户月订单数 ####################
g = data_whole_year.groupby(['o_month'])
user_months = {}
for x, group in g:
    mon = 'mon_%s' % str(x)
    tmp_pd = pd.DataFrame()
    mon_num = 'oder_num_month_%s' % str(x)
    tmp_pd[mon_num] = group.groupby('user_id').agg(['count']).values[:, :1].flatten().tolist()
    tmp_pd['user_id'] = group.groupby('user_id').groups.keys()
    user_months[mon] = tmp_pd

##################### 生成用户月目标订单数 ####################
g = data_whole_year.groupby(['o_month'])
user_months_target = {}
for x, group in g:
    mon = 'mon_%s' % str(x)
    tmp_pd = pd.DataFrame()
    
    mon_target_num = 'target_oder_num_month_%s' % str(x)
    g_target = group.groupby(['user_id','cate']).agg(['count']).reset_index()
    g_target = g_target.loc[(g_target['cate']==101) | (g_target['cate']==30),:]
    tmp_pd[mon_target_num] = g_target.groupby('user_id').agg(['sum']).values[:, 2:3].flatten().tolist()
    tmp_pd['user_id'] = g_target.groupby('user_id').groups.keys()
    user_months_target[mon] = tmp_pd



C:\Anaconda3\lib\site-packages\pandas\core\base.py:331: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return self.obj.drop(self.exclusions, axis=1)


In [200]:
user_months_num = pd.DataFrame()
user_months_num['user_id'] = jdata_user_info_all['user_id']
for index, m in enumerate(user_months):
    user_months_num = pd.merge(user_months_num, user_months[m], on='user_id', how='left')
    
for index, m in enumerate(user_months_target):
    user_months_num = pd.merge(user_months_num, user_months_target[m], on='user_id', how='left')
    
user_months_num.fillna(0,inplace=True)

In [201]:
#目标品类只购买一次的要去掉
row,column = user_months_num.shape
user_months_num['target_sum'] = 0
for i in range(row):
    user_months_num.iloc[i,-1] = user_months_num.iloc[i,13:].sum()

In [202]:
target_user_group = user_months_num[user_months_num['target_sum']>1]['user_id'].drop_duplicates().to_frame().reset_index().drop(['index'],axis=1)

In [203]:
#order data分几个数据集，分段,可配置，通过split进行配置
#split =1 每隔1个月分析1次
month =[5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4]
order_data_month_split={}
split = 1
tem=[]
for i in range(0,len(month),1):
    tem.append(month[i:i+split])
for j in range(0,len(tem)):
    if len(tem[j])==split:
        mon_start = 'mon_%s' % str(tem[j][0])
        order_data_month_split[mon_start] = pd.DataFrame()
        tmp_df = pd.DataFrame()
        for k in tem[j]:
            tmp_df = jdata_order_behavior[jdata_order_behavior['o_month'] == k].drop_duplicates()
            order_data_month_split[mon_start] = pd.concat([order_data_month_split[mon_start], tmp_df])           

In [204]:
#behavior data分几个数据集,可配置
behavior_data_month_split={}
tem=[]
for i in range(0,len(month),1):
    tem.append(month[i:i+split])
for j in range(0,len(tem)):
    if len(tem[j])==split:
        mon_start = 'mon_%s' % str(tem[j][0])
        behavior_data_month_split[mon_start] = pd.DataFrame()
        tmp_df = pd.DataFrame()
        for k in tem[j]:
            tmp_df = jdata_user_behavior[jdata_user_behavior['a_month'] == k].drop_duplicates()
            behavior_data_month_split[mon_start] = pd.concat([behavior_data_month_split[mon_start], tmp_df]) 

In [205]:
#comment分几个数据集,可配置
comment_data_month_split={}
tem=[]
for i in range(0,len(month),1):
    tem.append(month[i:i+split])
for j in range(0,len(tem)):
    if len(tem[j])==split:
        mon_start = 'mon_%s' % str(tem[j][0])
        comment_data_month_split[mon_start] = pd.DataFrame()
        tmp_df = pd.DataFrame()
        for k in tem[j]:
            tmp_df = jdata_user_comment_score[jdata_user_comment_score['month'] == k].drop_duplicates()
            comment_data_month_split[mon_start] = pd.concat([comment_data_month_split[mon_start], tmp_df]) 

In [206]:
#order data  cate 改为 0或者1
for index, m in enumerate(order_data_month_split):
        order_data_month_split[m].loc[(order_data_month_split[m]['cate']==101) | (order_data_month_split[m]['cate']==30),'cate']=1
        order_data_month_split[m].loc[(order_data_month_split[m]['cate']!=1),'cate']=0

In [207]:
#behavior data  cate 改为 0或者1
for index, m in enumerate(behavior_data_month_split):
        behavior_data_month_split[m].loc[(behavior_data_month_split[m]['cate']==101) | (behavior_data_month_split[m]['cate']==30),'cate']=1
        behavior_data_month_split[m].loc[(behavior_data_month_split[m]['cate']!=1),'cate']=0

In [208]:
#；平均购买间隔
'''
import math
data_whole_year['dayofyear'] = data_whole_year['o_date'].apply(lambda x: x.dayofyear)
data_whole_year.sort_values(by=['user_id','o_date'],ascending=[1,1],inplace=True)  
user_id_list = data_whole_year.loc[(data_whole_year['cate']==101)|(data_whole_year['cate']==30),'user_id'].drop_duplicates()

time_ave_diff = {}
for userid in user_id_list.tolist():
    temp_data_df = data_whole_year.loc[((data_whole_year['cate']==101)|(data_whole_year['cate']==30))&(data_whole_year['user_id']==userid),:]
    temp_group = temp_data_df.groupby(['user_id','o_date','o_id'])['dayofyear'].mean().to_frame().reset_index()
    temp_group['o_date_next'] = temp_group["o_date"].shift(1)  
    temp_group['time_diff'] = temp_group["o_date"]-temp_group["o_date_next"]
    temp_group['time_diff'] =  temp_group['time_diff'].apply(lambda x: x.days)
    temp_group = temp_group.fillna(0)
    if len(temp_group['time_diff'])==1:
        time_ave_diff[userid] = 0
    else:
        time_ave_diff[userid] = math.ceil(temp_group['time_diff'][1:].mean())
        

user_purchase_time_gap = pd.DataFrame([time_ave_diff]).T.reset_index()
user_purchase_time_gap.columns=['user_id','time_gap']

user_purchase_time_gap
'''

'\nimport math\ndata_whole_year[\'dayofyear\'] = data_whole_year[\'o_date\'].apply(lambda x: x.dayofyear)\ndata_whole_year.sort_values(by=[\'user_id\',\'o_date\'],ascending=[1,1],inplace=True)  \nuser_id_list = data_whole_year.loc[(data_whole_year[\'cate\']==101)|(data_whole_year[\'cate\']==30),\'user_id\'].drop_duplicates()\n\ntime_ave_diff = {}\nfor userid in user_id_list.tolist():\n    temp_data_df = data_whole_year.loc[((data_whole_year[\'cate\']==101)|(data_whole_year[\'cate\']==30))&(data_whole_year[\'user_id\']==userid),:]\n    temp_group = temp_data_df.groupby([\'user_id\',\'o_date\',\'o_id\'])[\'dayofyear\'].mean().to_frame().reset_index()\n    temp_group[\'o_date_next\'] = temp_group["o_date"].shift(1)  \n    temp_group[\'time_diff\'] = temp_group["o_date"]-temp_group["o_date_next"]\n    temp_group[\'time_diff\'] =  temp_group[\'time_diff\'].apply(lambda x: x.days)\n    temp_group = temp_group.fillna(0)\n    if len(temp_group[\'time_diff\'])==1:\n        time_ave_diff[userid]

In [209]:
#数据清洗
#对一个商品购买和其他交互行为(浏览,点击,收藏等)时间差应该多于一天
#因为根据赛题,我们需要预测下一个月的购买情况,那么如果用户对某商品在同一月完成所有的交互行为(包括购买),
#无法从这种交易中指导未来的预测。


In [210]:
#准备数据集合，正负样本
#2016年7月-2017年3月 2017年4月作为区分正样本和负样本  找出用户ID
user_pos_neg_group={}
for month in [5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4]:
        mon = 'mon_%s' % str(month)
        tmp_df = pd.DataFrame()
        mon_data = jdata_order_behavior[jdata_order_behavior['o_month'] == month].drop_duplicates()
        unique_userID_positive = mon_data.loc[(mon_data['cate']==101)|(mon_data['cate']==30),'user_id'].drop_duplicates()
        unique_userID_positive = unique_userID_positive.to_frame().reset_index().drop(['index'],axis=1)
        unique_userID_negative = mon_data.loc[(mon_data['cate']!=101)&(mon_data['cate']!=30),'user_id'].drop_duplicates()
        unique_userID_negative = unique_userID_negative.to_frame().reset_index().drop(['index'],axis=1)
        
        unique_userID_positive['target'] =1
        unique_userID_negative['target'] =0
        unique_userID = unique_userID_positive.append(unique_userID_negative)
        user_pos_neg_group[mon] = unique_userID
        

In [230]:
jdata_user_info_all_dict = {}
for index, m in enumerate(behavior_data_month_split):
    
    #消费平均价格
    users_price_df = pd.DataFrame()
    g_users = behavior_data_month_split[m].groupby(['user_id'])
    all_users = g_users.groups.keys()
    users_price_df['user_id'] = all_users
    users_price_df['avg_price'] = g_users.agg(['mean']).values[:,4:5].flatten().tolist()
    
    #浏览量和关注量
    
    behavior_data_month_split[m].loc[(behavior_data_month_split[m]['cate']==101) | (behavior_data_month_split[m]['cate']==30),'cate']=1
    behavior_data_month_split[m].loc[(behavior_data_month_split[m]['cate']!=1),'cate']=0
    jdata_behavior_between_total = behavior_data_month_split[m].groupby(['user_id','cate','a_type'])['a_num'].sum().to_frame().reset_index()
    jdata_behavior_between_surf_target =jdata_behavior_between_total.loc[(jdata_behavior_between_total['cate']==1) & (jdata_behavior_between_total['a_type']==1),:].reset_index().drop(['index'],axis=1)
    jdata_behavior_between_surf_target.columns=['user_id','cate','a_type','surf_num']

    jdata_behavior_between_add_target =jdata_behavior_between_total.loc[(jdata_behavior_between_total['cate']==1) & (jdata_behavior_between_total['a_type']==2),:].reset_index().drop(['index'],axis=1)
    jdata_behavior_between_add_target.columns=['user_id','cate','a_type','add_num']

    jdata_behavior_between_surf_relate =jdata_behavior_between_total.loc[(jdata_behavior_between_total['cate']==0) & (jdata_behavior_between_total['a_type']==1),:].reset_index().drop(['index'],axis=1)
    jdata_behavior_between_surf_relate.columns=['user_id','cate','a_type','surf_relate_num']

    jdata_behavior_between_add_relate =jdata_behavior_between_total.loc[(jdata_behavior_between_total['cate']==0) & (jdata_behavior_between_total['a_type']==2),:].reset_index().drop(['index'],axis=1)
    jdata_behavior_between_add_relate.columns=['user_id','cate','a_type','add_relate_num']
    
    
    #comment_num(评论数),has_bad_comment(是否有差评),bad_comment_rate(差评率)
    #用户的总评论数
    user_comment_total = comment_data_month_split[m][comment_data_month_split[m]['score_level']!=-1].groupby(['user_id'])['score_level'].count().to_frame().reset_index()
    user_comment_total.columns=['user_id','total_comments']
    

    #是否有差评
    jdata_bad_comment = comment_data_month_split[m][comment_data_month_split[m]['score_level']==3]
    jdata_user_bad_comment_list = pd.merge(order_data_month_split[m],jdata_bad_comment,on='o_id',how='left')
    jdata_bad_comment_unique_userid = jdata_user_bad_comment_list.loc[(jdata_user_bad_comment_list['score_level']==3) & (jdata_user_bad_comment_list['cate']==1),: ]['user_id_x'].drop_duplicates().to_frame().reset_index().drop(['index'],axis=1)
    jdata_bad_comment_unique_userid['has_bad_comments'] = 1
    jdata_bad_comment_unique_userid.columns=['user_id','has_bad_comments']
    
    
    
    #用户维度合并
    jdata_user_info = jdata_user_info_all
    #匹配平均消费价格
    jdata_user_info = pd.merge(jdata_user_info,users_price_df,on='user_id',how='left')
    #匹配总订单量
    jdata_user_info = pd.merge(jdata_user_info,users_pd,on='user_id',how='left')
    #匹配目标和相关产品订单量
    jdata_user_info = pd.merge(jdata_user_info,user_months_target[m],on='user_id',how='left')
    jdata_user_info = pd.merge(jdata_user_info,user_months[m],on='user_id',how='left')
    #匹配平均购买频次
    jdata_user_info = pd.merge(jdata_user_info,user_purchase_time_gap.loc[:,['user_id','time_gap']],on='user_id',how='left')
    #匹配浏览量和关注量
    jdata_user_info = pd.merge(jdata_user_info,jdata_behavior_between_surf_target.loc[:,['user_id','surf_num']],on='user_id',how='left')
    jdata_user_info = pd.merge(jdata_user_info,jdata_behavior_between_add_target.loc[:,['user_id','add_num']],on='user_id',how='left')
    
    #匹配评论量和差评量
    jdata_user_info = pd.merge(jdata_user_info,user_comment_total,on='user_id',how='left')
    jdata_user_info = pd.merge(jdata_user_info,jdata_bad_comment_unique_userid,on='user_id',how='left')
        
    
    
    jdata_user_info = pd.merge(target_user_group,jdata_user_info,on='user_id',how='left')
    
    
   
    #合并后
    jdata_user_info =jdata_user_info.drop(['sku_id'],axis=1)
    jdata_user_info.columns=['user_id','age','sex','user_level','o_area','price','total_order_num','target_order_num_month','order_num_month','time_gap','surf_time','add_time','total_comments','has_bad_comment']
    
    #jdata_user_info = jdata_user_info[(jdata_user_info['time_gap']<30) & (jdata_user_info['time_gap']!=0)]
    
    
    
    #关联匹配后的数据再清洗
    jdata_user_info = jdata_user_info.fillna(0)
    import math 
    #年龄用户均值替代 -1的
    jdata_user_info['age'] = jdata_user_info['age'].replace(-1, math.floor(jdata_user_info['age'].mean()))




    import numpy as np
    def feature_scaling(X, axis=0):
        new = X - np.mean(X, axis=0)
        return new / np.std(new, axis=0) 

    #浏览转化率
    jdata_user_info['surf_ratio'] = jdata_user_info['target_order_num_month']/user['surf_time']
    jdata_user_info['surf_ratio'] = jdata_user_info['surf_ratio'].replace(np.inf,0)
    jdata_user_info['surf_ratio'] = jdata_user_info['surf_ratio'].replace(np.nan,0)
 
    #Y值 打上标签

    if int(m.split('_')[-1]) ==(4-split+1):
        jdata_user_info['target'] = 0
    elif int(m.split('_')[-1]) >=(12-split+1):
        jdata_user_info =  pd.merge(jdata_user_info,user_pos_neg_group['mon_'+str((int((m.split('_')[-1]))+split)%12)],on='user_id',how='left')
    else:
        jdata_user_info =  pd.merge(jdata_user_info,user_pos_neg_group['mon_'+str(int((m.split('_')[-1]))+split)],on='user_id',how='left')



    #得到训练集 4月预测  5月的

      
    user_train_data1 = jdata_user_info.loc[(jdata_user_info['target']>=0),:]
    

    jdata_user_info_all_dict[m] = user_train_data1
    
    

In [247]:
#数据清洗、特征工程 TestDataset
#4月份为预测集合
user_test_data = pd.DataFrame()
user_test_data = pd.concat([user_test_data, jdata_user_info_all_dict['mon_4']])

#其他月份均为训练集
train = pd.DataFrame()
for index, m in enumerate(jdata_user_info_all_dict):
    if m != 'mon_4':
        train = pd.concat([train, jdata_user_info_all_dict[m]])

In [249]:
train_positve = train[train['target']==1].sample(n=110000) 

In [250]:
train=  pd.concat([train[train['target']==0], train_positve])

In [251]:
train['target'].value_counts()

0.0    110165
1.0    110000
Name: target, dtype: int64

In [270]:
user_test_data.head()

,user_id,age,sex,user_level,o_area,price,total_order_num,target_order_num_month,order_num_month,time_gap,surf_time,add_time,total_comments,has_bad_comment,surf_ratio,target
0,1,3,2,3,14,1.0,6,2.0,2.0,22.0,11.0,0.0,0.0,0.0,0.090909,0
1,3,3,0,5,26,1.0,18,1.0,1.0,43.0,8.0,0.0,0.0,0.0,0.000000,0
2,6,2,2,2,13,1.0,3,1.0,1.0,105.0,5.0,0.0,0.0,0.0,0.000000,0
3,7,3,2,4,5,1.0,10,1.0,1.0,23.0,7.0,0.0,0.0,0.0,0.027778,0
4,8,2,2,2,27,0.0,3,0.0,0.0,54.0,0.0,0.0,0.0,0.0,0.000000,0


In [256]:
#归一化处理
#特征离散化处理
#年龄分段
    
age_df = pd.get_dummies(train["age"], prefix="age")  
sex_df = pd.get_dummies(train["sex"], prefix="sex")  
user_lv_df = pd.get_dummies(train["user_level"], prefix="user_lv") 
#area_df = pd.get_dummies(jdata_user_info["o_area"], prefix="area") 

user = pd.concat([train, age_df, sex_df, user_lv_df], axis=1)  
        
user['price']=feature_scaling(user['price'])
user['total_order_num']=feature_scaling(user['total_order_num'])
user['target_order_num_month']=feature_scaling(user['target_order_num_month'])
#user['order_num_month']=feature_scaling(user['order_num_month'])
user['time_gap']=feature_scaling(user['time_gap'])
user['surf_time']=feature_scaling(user['surf_time'])
user['add_time']=feature_scaling(user['add_time'])

In [ ]:
age_df = pd.get_dummies(user_test_data["age"], prefix="age")  
sex_df = pd.get_dummies(user_test_data["sex"], prefix="sex")  
user_lv_df = pd.get_dummies(user_test_data["user_level"], prefix="user_lv") 
#area_df = pd.get_dummies(jdata_user_info["o_area"], prefix="area") 

user_test_data = pd.concat([user_test_data, age_df, sex_df, user_lv_df], axis=1)  
        
user_test_data['price']=feature_scaling(user_test_data['price'])
user_test_data['total_order_num']=feature_scaling(user_test_data['total_order_num'])
user_test_data['target_order_num_month']=feature_scaling(user_test_data['target_order_num_month'])
#user['order_num_month']=feature_scaling(user['order_num_month'])
user_test_data['time_gap']=feature_scaling(user_test_data['time_gap'])
user_test_data['surf_time']=feature_scaling(user_test_data['surf_time'])
user_test_data['add_time']=feature_scaling(user_test_data['add_time'])

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler

dataset12_y = user.target
dataset12_x = user.drop(['user_id','target'],axis=1)

dataset3_preds['user_id'] = user_test_data[['user_id']]
dataset3_x = user_test_data.drop(['user_id','target'],axis=1) 


params={'booster':'gbtree',
	    'objective': 'rank:pairwise',
	    'eval_metric':'auc',
	    'gamma':0.1,
	    'min_child_weight':1.1,
	    'max_depth':5,
	    'lambda':10,
	    'subsample':0.7,
	    'colsample_bytree':0.7,
	    'colsample_bylevel':0.7,
	    'eta': 0.01,
	    'tree_method':'exact',
	    'seed':0,
	    'nthread':12
	    }


dataset12 = xgb.DMatrix(dataset12_x,label=dataset12_y)
dataset3 = xgb.DMatrix(dataset3_x)

watchlist = [(dataset12,'train')]
model = xgb.train(params,dataset12,num_boost_round=3500,evals=watchlist)


dataset3_preds['target'] = model.predict(dataset3)
dataset3_preds.sort_values(by=['target'],ascending=[0],inplace=True)


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0]	train-auc:0.712852
[1]	train-auc:0.730887
[2]	train-auc:0.731997
[3]	train-auc:0.731407
[4]	train-auc:0.733149
[5]	train-auc:0.733585
[6]	train-auc:0.732738
[7]	train-auc:0.732464
[8]	train-auc:0.731593
[9]	train-auc:0.729494
[10]	train-auc:0.731171
[11]	train-auc:0.729246
[12]	train-auc:0.731297
[13]	train-auc:0.731642
[14]	train-auc:0.730857
[15]	train-auc:0.730319
[16]	train-auc:0.730943
[17]	train-auc:0.731821
[18]	train-auc:0.731176
[19]	train-auc:0.731083
[20]	train-auc:0.73038
[21]	train-auc:0.731221
[22]	train-auc:0.730317
[23]	train-auc:0.731986
[24]	train-auc:0.733147
[25]	train-auc:0.733676
[26]	train-auc:0.733438
[27]	train-auc:0.733463
[28]	train-auc:0.733534
[29]	train-auc:0.734048
[30]	train-auc:0.733988
[31]	train-auc:0.734312
[32]	train-auc:0.734478
[33]	train-auc:0.733941
[34]	train-auc:0.734418
[35]	train-auc:0.734996
[36]	train-auc:0.735564
[37]	train-auc:0.735448
[38]	train-auc:0.735681
[39]	train-auc:0.735699
[40]	train-auc:0.735327
[41]	train-auc:0.735259
[42

ValueError: feature_names mismatch: ['age', 'sex', 'user_level', 'o_area', 'price', 'total_order_num', 'target_order_num_month', 'order_num_month', 'time_gap', 'surf_time', 'add_time', 'total_comments', 'has_bad_comment', 'surf_ratio', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'sex_0', 'sex_1', 'sex_2', 'user_lv_1', 'user_lv_2', 'user_lv_3', 'user_lv_4', 'user_lv_5'] ['age', 'sex', 'user_level', 'o_area', 'price', 'total_order_num', 'target_order_num_month', 'order_num_month', 'time_gap', 'surf_time', 'add_time', 'total_comments', 'has_bad_comment', 'surf_ratio']
expected age_6, sex_1, user_lv_5, user_lv_3, age_3, age_5, user_lv_2, age_2, sex_0, user_lv_4, age_1, sex_2, user_lv_1, age_4 in input data

In [264]:
import datetime
times = [datetime.datetime(2017, 5, 1) for i in range(50000)]

test = pd.DataFrame()
# 根据年购买数量排序后截取前50000个user
test['user_id'] = dataset3_preds['user_id'][:50000]
test['pred_date'] = times

test.to_csv('test.csv', index=False)

In [ ]:
dataset3_preds

In [265]:
#save feature score
feature_score = model.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
    
with open('xgb_feature_score.csv','w') as f:
    f.writelines("feature,score\n")
    f.writelines(fs)